In [1]:
import gradio as gr
from langchain_openai import AzureChatOpenAI
import os

# Set up your Azure OpenAI credentials


# Function to read a file and return its content
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Define the initial system prompt
intro_user_prompt = read_file('../Prompts/SystemPrompt-VSC-BusinesCentral-TestRunner.txt')

# Define the initial user messages
user_messages = [
    read_file('../../BusinessCentral-AL-Test-Runner-App/src/codeunit/WebApi.Codeunit.al'),
    read_file('../../VSC-BusinessCentral-Test-Runner/package.json'),
    read_file('../../VSC-BusinessCentral-Test-Runner/src/apiClient.ts'),
    read_file('../../VSC-BusinessCentral-Test-Runner/src/project.ts'),
    read_file('../../VSC-BusinessCentral-Test-Runner/src/testRunner.ts'),
    read_file('../../VSC-BusinessCentral-Test-Runner/src/testViewProviders.ts'),
    read_file('../../VSC-BusinessCentral-Test-Runner/src/extension.ts')
]

# Initialize chat history
chat_history = [{"role": "system", "content": "Du bist ein freundliche Assistent und ein Programmierspezialist, der in deutscher Sprache antwortet"},
                {"role": "user", "content": intro_user_prompt}]

# Add user messages to chat history
for msg in user_messages:
    chat_history.append({"role": "user", "content": msg})
    chat_history.append({"role": "assistant", "content": "Datei gespeichert und verstanden. Bitte fortfahren"})

# Function to generate a response from OpenAI
def generate_response(input_text, chat_history):
    chat_history.append({"role": "user", "content": input_text})
    response = AzureChatOpenAI.ChatCompletion.create(
        model="gpt-4o",
        messages=chat_history
    )
    assistant_message = response['choices'][0]['message']['content']
    chat_history.append({"role": "assistant", "content": assistant_message})
    return assistant_message, chat_history

# Function to save chat history
def save_chat_history(chat_history, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(str(chat_history))

# Function to load chat history
def load_chat_history(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return eval(file.read())

# Gradio interface functions
def chat(input_text):
    global chat_history
    assistant_message, chat_history = generate_response(input_text, chat_history)
    return assistant_message

def save_conv():
    global chat_history
    file_path = gr.FileDialog.save_file()
    save_chat_history(chat_history, file_path)

def load_conv():
    global chat_history
    file_path = gr.FileDialog.open_file()
    chat_history = load_chat_history(file_path)
    return chat_history

def delete_all():
    global chat_history
    chat_history = [{"role": "system", "content": system_prompt}]
    return chat_history

def delete_message(index):
    global chat_history
    del chat_history[index]
    return chat_history

# Create Gradio Interface
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    input_text = gr.Textbox(placeholder="Type your message here...")
    input_button = gr.Button("Submit")
    save_button = gr.Button("Save")
    load_button = gr.Button("Load")
    delete_all_button = gr.Button("Delete All")

    # Display chat history
    for i, message in enumerate(chat_history):
        if message["role"] == "user":
            chatbot.append(message["content"], "user")
        else:
            chatbot.append(message["content"], "assistant")
        delete_button = gr.Button("Delete", elem_id=f"delete_{i}")
        delete_button.click(delete_message, i, chatbot)

    input_button.click(chat, input_text, chatbot)
    save_button.click(save_conv)
    load_button.click(load_conv)
    delete_all_button.click(delete_all)

demo.launch()

/home/kosta/miniconda3/envs/langchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: 'Chatbot' object has no attribute 'append'

In [35]:
import gradio as gr
from langchain_openai import AzureChatOpenAI
from langchain.callbacks.base import BaseCallbackManager, BaseCallbackHandler
import asyncio
import pprint

# Custom Callback Handler for Gradio
class GradioCallbackHandler(BaseCallbackHandler):
    def __init__(self, queue):
        self.queue = queue

    async def on_llm_new_token(self, token, **kwargs) -> None:
        # Check if token is an instance of a specific class, e.g., AIMessage
        pprint.pprint("on_llm_new_token");
        pprint.pprint(token)
        if hasattr(token, 'content'):
            await self.queue.put(token.content)
        else:
            await self.queue.put(str(token))

# Function to read a file and return its content
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Define the queue to hold the streaming messages
queue = asyncio.Queue()

# Configure the callback manager for streaming
callback_manager = BaseCallbackManager([GradioCallbackHandler(queue)])

llm = AzureChatOpenAI(
    model="gpt-4o",
    streaming=True,
    callback_manager=callback_manager
)

# Define the initial system prompt
intro_user_prompt = read_file('../Prompts/SystemPrompt-VSC-BusinesCentral-TestRunner.txt')

# Define the initial user messages
user_messages = [
    read_file('../../BusinessCentral-AL-Test-Runner-App/src/codeunit/WebApi.Codeunit.al'),
    read_file('../../VSC-BusinessCentral-Test-Runner/package.json'),
    read_file('../../VSC-BusinessCentral-Test-Runner/src/apiClient.ts'),
    read_file('../../VSC-BusinessCentral-Test-Runner/src/project.ts'),
    read_file('../../VSC-BusinessCentral-Test-Runner/src/testRunner.ts'),
    read_file('../../VSC-BusinessCentral-Test-Runner/src/testViewProviders.ts'),
    read_file('../../VSC-BusinessCentral-Test-Runner/src/extension.ts')
]

# Initialize chat history
chat_history = [
    {"role": "system", "content": "Du bist ein freundlicher Assistent und ein Programmierspezialist, der in deutscher Sprache antwortet"},
    {"role": "user", "content": intro_user_prompt}
]

# Add user messages to chat history
for msg in user_messages:
    chat_history.append({"role": "user", "content": msg})
    chat_history.append({"role": "assistant", "content": "Datei gespeichert und verstanden. Bitte fortfahren"})

# Function to generate a response from OpenAI
async def generate_response(input_text, chat_history):
    chat_history.append({"role": "user", "content": input_text})
    response = await llm.invoke(chat_history)
    for token in response:
        await queue.put(token)
    await queue.put(None)  # Indicate the end of the response

# Gradio interface functions
async def chat(input_text, gr_chathistory):
    global chat_history
    response = ""
    task = asyncio.create_task(generate_response(input_text, chat_history))
    while True:
        token = await queue.get()
        if token is None:
            break
        response += token
        yield response
    chat_history.append({"role": "assistant", "content": response})

# Create Gradio Interface
with gr.Blocks() as demo:
    chat_interface = gr.ChatInterface(fn=chat, chatbot=gr.Chatbot(height=300))

    # Additional elements for file operations
    save_button = gr.Button("Save")
    load_button = gr.Button("Load")
    delete_all_button = gr.Button("Delete All")

    def save_conv():
        global chat_history
        file_path = gr.FileDialog.save_file()
        save_chat_history(chat_history, file_path)

    def load_conv():
        global chat_history
        file_path = gr.FileDialog.open_file()
        chat_history = load_chat_history(file_path)
        return [(msg["content"], msg["role"]) for msg in chat_history if msg["role"] in ["user", "assistant"]]

    def delete_all():
        global chat_history
        chat_history = [{"role": "system", "content": intro_user_prompt}]
        return [(msg["content"], msg["role"]) for msg in chat_history if msg["role"] in ["user", "assistant"]]

    save_button.click(save_conv)
    load_button.click(load_conv, None, chat_interface)
    delete_all_button.click(delete_all, None, chat_interface)

demo.launch()

Running on local URL:  http://127.0.0.1:7888

To create a public link, set `share=True` in `launch()`.


Error in callback coroutine: AttributeError("'str' object has no attribute 'content'")


'on_llm_new_token'
''


Error in callback coroutine: AttributeError("'str' object has no attribute 'content'")
Error in callback coroutine: AttributeError("'str' object has no attribute 'content'")
Error in callback coroutine: AttributeError("'str' object has no attribute 'content'")
Error in callback coroutine: AttributeError("'str' object has no attribute 'content'")
Error in callback coroutine: AttributeError("'str' object has no attribute 'content'")
Error in callback coroutine: AttributeError("'str' object has no attribute 'content'")
Error in callback coroutine: AttributeError("'str' object has no attribute 'content'")
Error in callback coroutine: AttributeError("'str' object has no attribute 'content'")
Error in callback coroutine: AttributeError("'str' object has no attribute 'content'")
Error in callback coroutine: AttributeError("'str' object has no attribute 'content'")
Error in callback coroutine: AttributeError("'str' object has no attribute 'content'")
Error in callback coroutine: AttributeError

'on_llm_new_token'
'Die'
'on_llm_new_token'
' Datei'
'on_llm_new_token'
' ist'
'on_llm_new_token'
' geladen'
'on_llm_new_token'
' und'
'on_llm_new_token'
' verstanden'
'on_llm_new_token'
'.'
'on_llm_new_token'
' Gibt'
'on_llm_new_token'
' es'
'on_llm_new_token'
' eine'
'on_llm_new_token'
' bestimmte'
'on_llm_new_token'
' Frage'
'on_llm_new_token'
' oder'
'on_llm_new_token'
' eine'
'on_llm_new_token'
' Änderung'
'on_llm_new_token'
','
'on_llm_new_token'
' die'
'on_llm_new_token'
' du'
'on_llm_new_token'
' durchführen'
'on_llm_new_token'
' möchtest'
'on_llm_new_token'
'?'
'on_llm_new_token'
''
